In [9]:
import os
import dotenv

dotenv.load_dotenv()
GRID_API_KEY = os.getenv("GRID_API_KEY")
LOL_TITLE_ID = 3
STATIC_AAPI_URL = "https://api-op.grid.gg/central-data/graphql/"


In [10]:
import requests
def fetch_from_grid(
        query:str,
        url:str = "https://api-op.grid.gg/central-data/graphql/",
        api_key:str = GRID_API_KEY,
) -> requests.Response:
    headers = {
        "x-api-key": api_key
    }

    req_json = {
        "query": query
    }

    res = requests.post(
        url,
        headers=headers,
        json = req_json
    )

    return res.json()

# grab schema


In [ ]:
import json

body = """
{
  __schema {
    queryType {
      fields{
        name
        description
        args {
          name
          description
          type {
            name
            description
            kind
            ofType {
              name
              kind
            }
          }
        }
        type {
          name
          description
          kind
          ofType {
            name
            kind
          }
        }
      }
    }
  }
}
"""

res = fetch_from_grid(body, STATIC_AAPI_URL, GRID_API_KEY)

with open("central-data-query-schema.json", 'w') as f:
    json.dump(res, f, indent=2)

In [ ]:

body = """
{
    __schema {
        types {
            name
            description
            kind
            fields {
                name
                description
                type {
                    name
                    description
                    kind
                    ofType {
                        name
                        kind
                    }
                }
            }
            args {
                name
                description
                type {
                    name
                    kind
                    ofType {
                        name
                        kind
                    }
                }
            }
        }
    }
}
"""

res = fetch_from_grid(body, STATIC_AAPI_URL, GRID_API_KEY)

with open("central-data-types-schema.json", 'w') as f:
    json.dump(res, f, indent=2)

# STATIC DATA TEST

In [ ]:
from pprint import pprint

In [ ]:
body = """
{
    allSeries (
        filter: {
            title: 3
        }
    ) {
        totalCount,
        pageInfo{
          hasPreviousPage
          hasNextPage
          startCursor
          endCursor
        }
        edges{
          cursor
          node{
              id
              title {
                nameShortened
              }
              tournament {
                nameShortened
              }
              startTimeScheduled
              format {
                name
                nameShortened
              }
              teams {
                baseInfo {
                  name
                }
                scoreAdvantage
              }
          }
        }
      }
}
"""

res = fetch_from_grid(body, STATIC_AAPI_URL, GRID_API_KEY)
pprint(res)

# SERIES IN-GAME DATA TEST (this is for live data)


In [ ]:
AAPI_URL = "https://api-op.grid.gg/live-data-feed/series-state/graphql"


